In [ ]:
from autogluon.multimodal import MultiModalPredictor
import uuid
import pandas as pd

In [2]:
from torchvision.datasets import VisionDataset

from PIL import Image


def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')


class Caltech1(VisionDataset):
    def __init__(self, root, split='train', transform=None, target_transform=None, use_path=False):
        super(Caltech1, self).__init__(root, transform=transform, target_transform=target_transform)
        self.use_path = use_path

        self.split = split

        #path = "/content/Caltech101/" + self.split + ".txt"
        path = "./Caltech101/" + self.split + ".txt"

        self.data = list()
        self.labels = set()
        
        with open(path, 'r') as f:
            for line in f:
                #image_path = "/content/" + root + '/' + line.strip()
                image_path = "" + root + '/' + line.strip()

                label = line.strip().split('/')[0]
                if self.use_path:
                    image = image_path
                else:
                    image = pil_loader(image_path)
                if "BACKGROUND_Google" not in label:
                    self.labels.add(label)
                    self.data.append((image, list(self.labels).index(label)))

    def __getitem__(self, index):
        '''
        __getitem__ should access an element through its index
        Args:
            index (int): Index

        Returns:
            tuple: (sample, target) where target is class_index of the target class.
        '''

        image, label = self.data[index]

        # Applies preprocessing when accessing the image
        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        '''
        The __len__ method returns the length of the dataset
        It is mandatory, as this is used by several other components
        '''
        return len(self.data)


In [3]:
from torch.utils.data import Subset

DATA_DIR = 'Caltech101/101_ObjectCategories'

from sklearn.model_selection import train_test_split

train_ds = Caltech1(DATA_DIR, split='train', use_path=True)
test_dataset = Caltech1(DATA_DIR, split='test', use_path=True)

ids = [x for x in range(len(train_ds))]
labels_indexes = [y for (x,y) in train_ds]

train_indexes, val_indexes = train_test_split(ids, test_size=0.5, shuffle=True, stratify=labels_indexes)

train_dataset = Subset(train_ds, train_indexes)
val_dataset = Subset(train_ds, val_indexes)

# Check dataset sizes
print('Train Dataset: {}'.format(len(train_dataset)))
print('Valid Dataset: {}'.format(len(val_dataset)))
print('Test Dataset: {}'.format(len(test_dataset)))

Train Dataset: 2892
Valid Dataset: 2892
Test Dataset: 2893


In [4]:
train_list = [j for i, j in enumerate(train_ds) if i in train_indexes]
val_list = [j for i, j in enumerate(train_ds) if i in val_indexes]

train_df = pd.DataFrame(train_list, columns=['image', 'label'])
val_df = pd.DataFrame(val_list, columns=['image', 'label'])


In [10]:
model_path = f"./autogloun_model/"
predictor = MultiModalPredictor(label="label").load(path=f"{model_path}")

Load pretrained checkpoint: C:\Users\alcarachin\Desktop\autoML_test\caltech_test\autogloun_model\model.ckpt


In [8]:
model_path = f"./autogloun_model/"
predictor = MultiModalPredictor(label="label", path=model_path)
predictor.fit(
    train_data=train_df,
    time_limit=7200,
)

=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          8
Pytorch Version:    2.0.1+cpu
CUDA Version:       CUDA is not available
Memory Avail:       13.56 GB / 31.85 GB (42.6%)
Disk Space Avail:   776.77 GB / 940.03 GB (82.6%)
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	First 10 (of 48) unique label values:  [0, 1, 3, 2, 4, 6, 11, 5, 12, 14]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Epoch 0, global step 9: 'val_accuracy' reached 0.35200 (best 0.35200), saving model to 'C:\\Users\\alcarachin\\Desktop\\autoML_test\\caltech_test\\autogloun_model\\epoch=0-step=9.ckpt' as top 3


...

Epoch 2, global step 39: 'val_accuracy' reached 0.95000 (best 0.95000), saving model to 'C:\\Users\\alcarachin\\Desktop\\autoML_test\\caltech_test\\autogloun_model\\epoch=2-step=39.ckpt' as top 3
Start to fuse 3 checkpoints via the greedy soup algorithm.


Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("C:\Users\alcarachin\Desktop\autoML_test\caltech_test\autogloun_model")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




In [9]:
scores = predictor.evaluate(val_df, metrics=["accuracy"])
print('Top-1 test acc: %.3f' % scores["accuracy"])

Predicting: 0it [00:00, ?it/s]

Top-1 test acc: 0.956


In [13]:
predictor.predict(val_df[100:105])

Predicting: 0it [00:00, ?it/s]

100    1
101    1
102    1
103    1
104    1
Name: label, dtype: int64

In [ ]:
import shap
import matplotlib.pyplot as plt
import pandas as pd

def wrapped_model(x):
    x = pd.DataFrame(x)
    preds = predictor.predict(x).to_numpy()
        
    return preds

test_data = TabularDataset('Test.csv')

to_be_explained = pd.DataFrame(test_data).drop('label', axis=1).to_numpy()[0]
explainer = shap.KernelExplainer(wrapped_model, pd.read_csv('./Train.csv').drop('label', axis=1).sample(n=100))
shap_values = explainer.shap_values(to_be_explained)
relevance = abs(shap_values.ravel())